# YouTube trending videos

## Etap 6 - Klasyfikator i wiedza dla klienta


### Wybór miar oceny klasyfikatorów

### Wybór pierwszego klasyfikatora

### Uczenie pierwszego klasyfikatora

### Testowanie pierwszego klasyfikatora

### Interpretacja predykcji pierwszego klasyfikatora


### Wybór drugiego klasyfikatora

### Uczenie drugiego klasyfikatora

### Testowanie drugiego klasyfikatora

### Interpretacja predykcji drugiego klasyfikatora

### Porównanie wyników klasyfikatorów

### Wiedza dla klienta